# CIA Factbook Data

Working with data from the CIA World Factbook https://www.cia.gov/library/publications/the-world-factbook/, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like:

* population - The population as of 2015.
* population_growth - The annual population growth rate, as a * percentage.
* area - The total land and water area.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Overview of Database and Data

In [2]:
%%sql
SELECT * FROM sqlite_master
WHERE type ='table'

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT * FROM facts 
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Stats

In [4]:
%%sql
SELECT 
    MIN(population), 
    MAX(population), 
    MIN(population_growth), 
    MAX(population_growth)
FROM facts

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


## Identifying outliers

Identifying countries with population 0 and population of 7256490011

In [5]:
%%sql
SELECT * FROM facts
WHERE population == (
    SELECT MIN(population) FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql
SELECT * FROM facts
WHERE population == (
    SELECT MAX(population) FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Exploring average Population and Area

In [7]:
%%sql
SELECT AVG(population), AVG(area) FROM facts

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


## Finding Densely Populated Countries

In [8]:
%%sql
SELECT * FROM facts
WHERE population > (
    SELECT AVG(population) FROM facts
) AND area < (
    SELECT AVG(area) FROM facts
)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


# Next steps

* Which countries have the highest ratios of water to land? Which countries have more water than land?
* Which countries will add the most people to their population next year?
* Which countries have a higher death rate than birth rate?

## Countries with highest ratio of water to land

In [9]:
%%sql
SELECT name, area_land, area_water, ROUND(CAST(area_water as float) / CAST(area_land as float),2) water_to_land
FROM facts
ORDER BY water_to_land DESC
LIMIT 5

Done.


name,area_land,area_water,water_to_land
British Indian Ocean Territory,60,54340,905.67
Virgin Islands,346,1564,4.52
Puerto Rico,8870,4921,0.55
"Bahamas, The",10010,3870,0.39
Guinea-Bissau,28120,8005,0.28


## Countries that will add the most people to their population next year

In [23]:
%%sql
SELECT name, 
    population, 
    population_growth, 
    (
        SELECT population * population_growth
        FROM facts
    ) - population AS pop_diff
FROM facts
ORDER BY pop_diff DESC
LIMIT 5

Done.


name,population,population_growth,pop_diff
Antarctica,0,None,75549273.44
Pitcairn Islands,48,0.0,75549225.44
Cocos (Keeling) Islands,596,0.0,75548677.44
Holy See (Vatican City),842,0.0,75548431.44
Niue,1190,0.03,75548083.44


In [26]:
%%sql
SELECT name, 
    population, 
    population_growth, 
    population * population_growth new_pop     
FROM facts
ORDER BY new_pop DESC
LIMIT 5

Done.


name,population,population_growth,new_pop
World,7256490011,1.08,7837009211.88
India,1251695584,1.22,1527068612.48
China,1367485388,0.45,615368424.6
Nigeria,181562056,2.45,444827037.20000005
Pakistan,199085847,1.46,290665336.62


Above coding error??

## Countries with a higher death rate than birth rate

In [16]:
%%sql
SELECT name, birth_rate, death_rate FROM facts
WHERE birth_rate < death_rate

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


In [18]:
%%sql
SELECT name, birth_rate, death_rate, ROUND(death_rate - birth_rate, 2) death_birth_diff FROM facts
WHERE birth_rate < death_rate
ORDER BY death_birth_diff DESC

Done.


name,birth_rate,death_rate,death_birth_diff
Bulgaria,8.92,14.44,5.52
Serbia,9.08,13.66,4.58
Latvia,10.0,14.31,4.31
Lithuania,10.1,14.27,4.17
Ukraine,10.72,14.46,3.74
Hungary,9.16,12.73,3.57
Germany,8.47,11.42,2.95
Slovenia,8.42,11.37,2.95
Romania,9.14,11.9,2.76
Croatia,9.45,12.18,2.73
